In [1]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [2]:
# help the interpreter find the fuzzy_artmap module 
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# scale the data to [0,1]
scaled_data = scaler.fit_transform(data.data)

In [4]:
from sklearn.model_selection import train_test_split
# Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(scaled_data, data.target, test_size=0.2)


In [5]:
from fuzzy_artmap import FuzzyArtMap
# create the Fuzzy ARTMAP instance
fuzzy_artmap = FuzzyArtMap(baseline_vigilance=0.95, debugging=True)


In [6]:
import torch
import numpy as np
# re-shape the labels to be one per row, instead of one per column
y_train = y_train.reshape(-1,1)
y_train = np.clip(y_train, 0.0, 1.0)

y_test = y_test.reshape(-1,1)
y_test = np.clip(y_test, 0.0, 1.0)

# Hard limit the floating point to [0.0, 1.0] and Complement encode the values ...
X_train = np.clip(X_train, 0.0, 1.0)
X_train = FuzzyArtMap.complement_encode(torch.from_numpy(X_train))

X_test = np.clip(X_test, 0.0, 1.0)
X_test = FuzzyArtMap.complement_encode(torch.from_numpy(X_test))

# ... and the labels
y_train = FuzzyArtMap.complement_encode(torch.from_numpy(y_train))
y_test = FuzzyArtMap.complement_encode(torch.from_numpy(y_test))

In [7]:
fuzzy_artmap.fit(X_train, y_train)

FuzzyArtMap(baseline_vigilance=0.95, debugging=True)

In [8]:
from collections import Counter
def test_predictions(fuzzy_artmap, test_set, test_labels):
    """Count the True Positives (TP), True Negatives (TN), False Positives (FP), and False Negatives (FN) for predictions made by the model"""
    predictions = fuzzy_artmap.predict(test_set)
    
    accuracy_counter = Counter({"TP": 0, "TN": 0, "FP": 0, "FN": 0})
    for prediction_index, prediction in enumerate(predictions):
        class_vector = test_labels[prediction_index]
        if class_vector[0].item():
            if prediction[0][0].item():
                update = {"TP": 1}
            else:
                update = {"FN": 1}
        else:
            if prediction[0][0].item():
                update = {"FP": 1}
            else:
                update = {"TN": 1}
        accuracy_counter.update(update)
    print(accuracy_counter)
    return accuracy_counter

In [9]:
def calculate_metrics(accuracy_data):
    """Calculate accuracy, precision, recall, and speed metrics given the accuracy data"""
    number_of_samples = sum(accuracy_data.values())
    accuracy = (accuracy_data["TP"] + accuracy_data["TN"]) / number_of_samples
    precision = accuracy_data["TP"] / (accuracy_data["TP"] + accuracy_data["FP"])
    recall = accuracy_data["TP"] / (accuracy_data["TP"] + accuracy_data["FN"])
    print(f"accuracy: {accuracy}\nprecision: {precision}\nrecall: {recall}\ntotal samples:{number_of_samples}")


In [10]:
accuracy_data = test_predictions(fuzzy_artmap, X_test, y_test)

Counter({'TP': 72, 'TN': 39, 'FP': 2, 'FN': 1})


In [11]:
calculate_metrics(accuracy_data)

accuracy: 0.9736842105263158
precision: 0.972972972972973
recall: 0.9863013698630136
total samples:114
